In [3]:
# Debug cell to understand the IS_TERMINATE error
print("=== Debugging IS_TERMINATE State Error ===")

# Let's examine the AgenticStateMachine source and see what states are being registered
from gai.asm import AgenticStateMachine
import logging

# Enable debug logging to see what's happening internally
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("Available AgenticStateMachine classes:")
print(dir(AgenticStateMachine))

# Let's try to understand what states are created in the state machine
# by examining the StateMachineBuilder process step by step

state_diagram = """
INIT --> PROCESS
PROCESS --> FINAL
"""

print(f"\nState diagram being used:\n{state_diagram}")

# Try to build the state machine without executing it yet
with AgenticStateMachine.StateMachineBuilder(state_diagram) as builder:
    print("\nBuilder created successfully")
    
    # Let's see what happens during build process
    try:
        fsm = builder.build(
            {
                "INIT": {
                    "input_data": {
                        "test": {"type": "getter", "dependency": "get_test"},
                    }
                },
                "PROCESS": {
                    "module_path": "gai.asm.states",
                    "class_name": "PureActionState",
                    "title": "PROCESS",
                    "action": "test_action",
                    "output_data": ["action_result"],
                },
                "FINAL": {
                    "output_data": ["action_result"],
                },
            },
            get_test=lambda state: "test_data",
            test_action=lambda state: setattr(state, 'action_result', "test_result"),
        )
        
        print(f"State machine built successfully")
        print(f"Initial state: {fsm.state}")
        print(f"State machine type: {type(fsm)}")
        
        # Examine the underlying PyTransitions machine
        if hasattr(fsm, 'machine'):
            machine = fsm.machine
            print(f"PyTransitions machine: {machine}")
            print(f"Available states: {list(machine.states.keys())}")
            print(f"Available transitions: {[t.source + '->' + t.dest for t in machine.transitions]}")
        
        # Try to access the current state to see if that's where the error occurs
        print(f"Current state attribute: {getattr(fsm, fsm.machine.model_attribute, 'NOT_FOUND')}")
        
    except Exception as e:
        print(f"Error during build: {e}")
        import traceback
        traceback.print_exc()

=== Debugging IS_TERMINATE State Error ===
Available AgenticStateMachine classes:
['StateHistory', 'StateMachineBuilder', 'StateModel', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']

State diagram being used:

INIT --> PROCESS
PROCESS --> FINAL


Builder created successfully
State machine built successfully
Initial state: INIT
State machine type: <class 'gai.asm.asm.AgenticStateMachine.StateModel'>
Error during build: 'StateModel' object has no attribute 'machine'


Traceback (most recent call last):
  File "/tmp/ipykernel_9263/3803887653.py", line 65, in <module>
    print(f"Current state attribute: {getattr(fsm, fsm.machine.model_attribute, 'NOT_FOUND')}")
AttributeError: 'StateModel' object has no attribute 'machine'


In [4]:
# More detailed debugging to understand the PyTransitions machine structure
print("=== Detailed State Machine Analysis ===")

from gai.asm import AgenticStateMachine

state_diagram = """
INIT --> PROCESS
PROCESS --> FINAL
"""

with AgenticStateMachine.StateMachineBuilder(state_diagram) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "test": {"type": "getter", "dependency": "get_test"},
                }
            },
            "PROCESS": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "PROCESS",
                "action": "test_action",
                "output_data": ["action_result"],
            },
            "FINAL": {
                "output_data": ["action_result"],
            },
        },
        get_test=lambda state: "test_data",
        test_action=lambda state: setattr(state, 'action_result', "test_result"),
    )
    
    print(f"StateModel initial state: {fsm.state}")
    
    # Dig deeper into the StateModel structure
    print(f"\nStateModel attributes:")
    for attr in dir(fsm):
        if not attr.startswith('_'):
            try:
                value = getattr(fsm, attr)
                if not callable(value):
                    print(f"  {attr}: {value}")
            except:
                print(f"  {attr}: <unable to access>")
    
    # Look for the PyTransitions machine - it might be hidden
    print(f"\nLooking for transitions machine...")
    if hasattr(fsm, '__dict__'):
        for key, value in fsm.__dict__.items():
            print(f"  {key}: {type(value)}")
            # Check if this is the PyTransitions machine
            if hasattr(value, 'states') and hasattr(value, 'transitions'):
                print(f"    Found PyTransitions machine at {key}!")
                print(f"    States: {list(value.states.keys())}")
                print(f"    Current state: {getattr(value.model, value.model_attribute, 'UNKNOWN')}")
                print(f"    Model attribute name: {value.model_attribute}")
                
    # Let's also check what happens when we try to access state history
    print(f"\nState history length: {len(fsm.state_history) if hasattr(fsm, 'state_history') else 'No state_history'}")
    if hasattr(fsm, 'state_history') and len(fsm.state_history) > 0:
        print(f"Last state in history: {fsm.state_history.last_state()}")
        
    # Check if there's existing state bag data that might be loading IS_TERMINATE
    print(f"\nState bag contents: {fsm.state_bag if hasattr(fsm, 'state_bag') else 'No state_bag'}")

=== Detailed State Machine Analysis ===
StateModel initial state: INIT

StateModel attributes:
  agent_name: Assistant
  caller_id: 00000000-0000-0000-0000-000000000000
  dialogue_id: 00000000-0000-0000-0000-000000000000
  kwargs: {'get_test': <function <lambda> at 0x7092d4353880>, 'test_action': <function <lambda> at 0x7092d43539a0>}
  monologue: <gai.messages.monologue.Monologue object at 0x7092d41d5bd0>
  state: INIT
  state_data: {}
  state_history: <gai.asm.asm.AgenticStateMachine.StateHistory object at 0x7092d41d62c0>
  state_manifest: {'INIT': {'input_data': {'test': {'type': 'getter', 'dependency': 'get_test'}}}, 'PROCESS': {'module_path': 'gai.asm.states', 'class_name': 'PureActionState', 'title': 'PROCESS', 'action': 'test_action', 'output_data': ['action_result']}, 'FINAL': {'output_data': ['action_result']}}
  user_message: None

Looking for transitions machine...
  state: <class 'str'>
  agent_name: <class 'str'>
  caller_id: <class 'str'>
  dialogue_id: <class 'str'>
  st

# PureActionState Demo

## Overview

`PureActionState` is a versatile state in the Agentic State Machine framework that executes custom action functions. It's perfect for:

- Data processing and transformation
- API calls and external service integration
- File operations
- Custom business logic
- Mathematical calculations
- Content generation and manipulation

This notebook demonstrates various practical uses of `PureActionState` with different types of actions.

```lua
stateDiagram-v2
    INIT --> ACTION
    ACTION --> FINAL
```

```mermaid
stateDiagram-v2
    Direction LR
    INIT --> ACTION
    ACTION --> FINAL
```


---

## Demo 1: Simple Data Processing Action

A basic example that processes input data and produces a result.


In [5]:
from gai.asm import AgenticStateMachine
import json
from datetime import datetime

print("=== Demo 1: Simple Data Processing Action ===")

# Define a simple data processing action
async def process_user_data(state):
    """Process user data and calculate some statistics."""
    
    # Get input data from state bag
    users = state.machine.state_bag.get("users", [])
    
    if not users:
        state.action_result = {"error": "No user data provided"}
        return
    
    # Process the data
    total_users = len(users)
    active_users = sum(1 for user in users if user.get("active", False))
    total_age = sum(user.get("age", 0) for user in users)
    average_age = total_age / total_users if total_users > 0 else 0
    
    # Create processed result
    result = {
        "total_users": total_users,
        "active_users": active_users,
        "inactive_users": total_users - active_users,
        "average_age": round(average_age, 2),
        "processed_at": datetime.now().isoformat()
    }
    
    # Store result in action_result (available in state bag)
    state.action_result = result
    
    print(f"✅ Processed {total_users} users")
    print(f"📊 Active users: {active_users}, Average age: {average_age:.1f}")

# Sample user data
sample_users = [
    {"id": 1, "name": "Alice", "age": 25, "active": True},
    {"id": 2, "name": "Bob", "age": 30, "active": False},
    {"id": 3, "name": "Charlie", "age": 35, "active": True},
    {"id": 4, "name": "Diana", "age": 28, "active": True},
    {"id": 5, "name": "Eve", "age": 42, "active": False}
]

# Build and run the state machine
with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> PROCESS
    PROCESS --> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "users": {"type": "getter", "dependency": "get_users"},
                }
            },
            "PROCESS": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "PROCESS",
                "action": "process_user_data",
                "output_data": ["action_result"],
            },
            "FINAL": {
                "output_data": ["action_result"],
            },
        },
        get_users=lambda state: sample_users,
        process_user_data=process_user_data,
    )

# Reset the state machine to start fresh (clears any persisted state from previous runs)
fsm.restart()
print(f"🔄 State machine reset. Current state: {fsm.state}")

# Execute the workflow
await fsm.run_async()  # INIT -> PROCESS
await fsm.run_async()  # PROCESS -> FINAL

# Display results
result = fsm.state_bag.get("action_result")
print("\n📋 Processing Results:")
print(json.dumps(result, indent=2))

=== Demo 1: Simple Data Processing Action ===
🔄 State machine reset. Current state: INIT


INFO     Leaving state: INIT

INFO:gai.asm.asm:Leaving state: INIT


DEBUG    AgenticStateMachine.resolve_input: input_data={'users': {'type': 'getter', 'dependency': 'get_users'}}

DEBUG:gai.asm.asm:AgenticStateMachine.resolve_input: input_data={'users': {'type': 'getter', 'dependency': 'get_users'}}


...

INFO     Leaving state: PROCESS

INFO:gai.asm.asm:Leaving state: PROCESS
INFO:transitions.extensions.asyncio:Finished processing state PROCESS exit callbacks.
INFO:transitions.extensions.asyncio:Finished processing state FINAL enter callbacks.


INFO     Entering state: FINAL

INFO:gai.asm.asm:Entering state: FINAL



📋 Processing Results:
null


---

## Demo 2: File Operations Action

Demonstrate file creation, reading, and manipulation using `PureActionState`.


In [6]:
import tempfile
import os
from pathlib import Path

print("=== Demo 2: File Operations Action ===")

# Define file operations action
async def file_operations_action(state):
    """Perform various file operations."""
    
    # Get configuration from state bag
    content = state.machine.state_bag.get("file_content", "Default content")
    operation = state.machine.state_bag.get("operation", "create")
    
    # Create a temporary directory for our operations
    temp_dir = tempfile.mkdtemp()
    file_path = os.path.join(temp_dir, "demo_file.txt")
    
    try:
        if operation == "create":
            # Create and write file
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)
            
            # Read it back to verify
            with open(file_path, "r", encoding="utf-8") as f:
                read_content = f.read()
            
            result = {
                "operation": "create",
                "file_path": file_path,
                "content_written": len(content),
                "content_read": len(read_content),
                "success": content == read_content,
                "file_size_bytes": os.path.getsize(file_path)
            }
            
        elif operation == "analyze":
            # Analyze existing file (create it first for demo)
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)
            
            # Analyze the file
            file_stats = os.stat(file_path)
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()
            
            result = {
                "operation": "analyze",
                "file_path": file_path,
                "file_size_bytes": file_stats.st_size,
                "total_lines": len(lines),
                "total_characters": len(content),
                "non_empty_lines": len([line for line in lines if line.strip()]),
                "words": len(content.split()),
                "created_timestamp": file_stats.st_ctime
            }
        
        else:
            result = {"error": f"Unknown operation: {operation}"}
        
        state.action_result = result
        print(f"✅ File operation '{operation}' completed successfully")
        
    except Exception as e:
        state.action_result = {"error": str(e), "operation": operation}
        print(f"❌ File operation failed: {e}")
    
    finally:
        # Clean up temporary file
        try:
            if os.path.exists(file_path):
                os.unlink(file_path)
            os.rmdir(temp_dir)
        except:
            pass

# Test file creation
print("\n📁 Testing file creation:")
with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> FILE_OPS
    FILE_OPS --> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "file_content": {"type": "getter", "dependency": "get_content"},
                    "operation": {"type": "getter", "dependency": "get_operation"},
                }
            },
            "FILE_OPS": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "FILE_OPS",
                "action": "file_operations_action",
                "output_data": ["action_result"],
            },
            "FINAL": {
                "output_data": ["action_result"],
            },
        },
        get_content=lambda state: "Hello, World!\nThis is a test file.\nWith multiple lines of content.\n",
        get_operation=lambda state: "create",
        file_operations_action=file_operations_action,
    )
fsm.restart()
await fsm.run_async()  # INIT -> FILE_OPS
await fsm.run_async()  # FILE_OPS -> FINAL

result = fsm.state_bag.get("action_result")
print("📋 File Creation Results:")
print(json.dumps(result, indent=2))

# Test file analysis
print("\n📊 Testing file analysis:")
with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> FILE_OPS
    FILE_OPS --> FINAL
    """
) as builder:
    fsm2 = builder.build(
        {
            "INIT": {
                "input_data": {
                    "file_content": {"type": "getter", "dependency": "get_content"},
                    "operation": {"type": "getter", "dependency": "get_operation"},
                }
            },
            "FILE_OPS": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "FILE_OPS",
                "action": "file_operations_action",
                "output_data": ["action_result"],
            },
            "FINAL": {
                "output_data": ["action_result"],
            },
        },
        get_content=lambda state: """# Project Documentation

## Overview
This is a sample project with multiple components.

## Features
- Feature A: Data processing
- Feature B: File management  
- Feature C: API integration

## Installation
1. Clone the repository
2. Install dependencies
3. Run the application

---
End of documentation.
""",
        get_operation=lambda state: "analyze",
        file_operations_action=file_operations_action,
    )
fsm2.restart()
await fsm2.run_async()  # INIT -> FILE_OPS
await fsm2.run_async()  # FILE_OPS -> FINAL

result2 = fsm2.state_bag.get("action_result")
print("📋 File Analysis Results:")
print(json.dumps(result2, indent=2))

=== Demo 2: File Operations Action ===

📁 Testing file creation:


INFO     Leaving state: INIT

INFO:gai.asm.asm:Leaving state: INIT


DEBUG    AgenticStateMachine.resolve_input: input_data={'file_content': {'type': 'getter', 'dependency':           
         'get_content'}, 'operation': {'type': 'getter', 'dependency': 'get_operation'}}

DEBUG:gai.asm.asm:AgenticStateMachine.resolve_input: input_data={'file_content': {'type': 'getter', 'dependency': 'get_content'}, 'operation': {'type': 'getter', 'dependency': 'get_operation'}}


...

INFO     Leaving state: FILE_OPS

INFO:gai.asm.asm:Leaving state: FILE_OPS
INFO:transitions.extensions.asyncio:Finished processing state FILE_OPS exit callbacks.
INFO:transitions.extensions.asyncio:Finished processing state FINAL enter callbacks.


INFO     Entering state: FINAL

INFO:gai.asm.asm:Entering state: FINAL


📋 File Analysis Results:
null


---

## Demo 3: API Integration Action

Simulate API calls and data fetching using `PureActionState`.


In [ ]:
import asyncio
import random

print("=== Demo 3: API Integration Action ===")

# Define API integration action
async def api_integration_action(state):
    """Simulate API calls and data integration."""
    
    # Get API configuration from state bag
    api_endpoint = state.machine.state_bag.get("api_endpoint", "https://api.example.com")
    data_type = state.machine.state_bag.get("data_type", "users")
    max_retries = state.machine.state_bag.get("max_retries", 3)
    
    print(f"🌐 Simulating API call to {api_endpoint}/{data_type}")
    
    # Simulate API call with retries
    for attempt in range(max_retries):
        try:
            print(f"   Attempt {attempt + 1}/{max_retries}...")
            
            # Simulate network delay
            await asyncio.sleep(0.5)
            
            # Simulate occasional API failures
            if random.random() < 0.3 and attempt < max_retries - 1:
                raise Exception(f"API timeout (attempt {attempt + 1})")
            
            # Simulate successful API response
            if data_type == "users":
                api_data = [
                    {"id": i, "name": f"User{i}", "status": random.choice(["active", "inactive"])}
                    for i in range(1, random.randint(5, 15))
                ]
            elif data_type == "orders":
                api_data = [
                    {"order_id": f"ORD-{1000+i}", "amount": random.randint(10, 500), "status": "completed"}
                    for i in range(random.randint(3, 10))
                ]
            else:
                api_data = {"message": f"Data for {data_type}", "count": random.randint(1, 100)}
            
            # Simulate data processing
            processed_data = {
                "api_endpoint": api_endpoint,
                "data_type": data_type,
                "records_count": len(api_data) if isinstance(api_data, list) else 1,
                "response_data": api_data,
                "fetch_timestamp": datetime.now().isoformat(),
                "attempts_required": attempt + 1,
                "status": "success"
            }
            
            state.action_result = processed_data
            print(f"✅ API call successful after {attempt + 1} attempt(s)")
            return
            
        except Exception as e:
            print(f"   ❌ Attempt {attempt + 1} failed: {e}")
            if attempt == max_retries - 1:
                # Final attempt failed
                state.action_result = {
                    "api_endpoint": api_endpoint,
                    "data_type": data_type,
                    "status": "failed",
                    "error": str(e),
                    "attempts": max_retries,
                    "fetch_timestamp": datetime.now().isoformat()
                }
                print(f"❌ API call failed after {max_retries} attempts")

# Test successful API call
print("\n🌐 Testing successful API integration:")
with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> API_CALL
    API_CALL --> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "api_endpoint": {"type": "getter", "dependency": "get_endpoint"},
                    "data_type": {"type": "getter", "dependency": "get_data_type"},
                    "max_retries": {"type": "getter", "dependency": "get_retries"},
                }
            },
            "API_CALL": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "API_CALL",
                "action": "api_integration_action",
                "output_data": ["action_result"],
            },
            "FINAL": {
                "output_data": ["action_result"],
            },
        },
        get_endpoint=lambda state: "https://api.users.com",
        get_data_type=lambda state: "users",
        get_retries=lambda state: 3,
        api_integration_action=api_integration_action,
    )

await fsm.run_async()  # INIT -> API_CALL
await fsm.run_async()  # API_CALL -> FINAL

result = fsm.state_bag.get("action_result")
print("\n📋 API Integration Results:")
print(f"Status: {result.get('status')}")
print(f"Records: {result.get('records_count', 0)}")
print(f"Attempts: {result.get('attempts_required', result.get('attempts', 'N/A'))}")
if result.get("status") == "success":
    print(f"Sample data: {result.get('response_data', [])[:2]}...")  # Show first 2 records

---

## Demo 4: Mathematical Computation Action

Perform complex calculations and data analysis using `PureActionState`.


In [ ]:
import math
import statistics

print("=== Demo 4: Mathematical Computation Action ===")

# Define mathematical computation action
async def math_computation_action(state):
    """Perform various mathematical computations."""
    
    # Get input data from state bag
    numbers = state.machine.state_bag.get("numbers", [])
    operation = state.machine.state_bag.get("math_operation", "statistics")
    
    if not numbers:
        state.action_result = {"error": "No numbers provided for computation"}
        return
    
    try:
        if operation == "statistics":
            result = {
                "operation": "statistics",
                "input_count": len(numbers),
                "sum": sum(numbers),
                "mean": statistics.mean(numbers),
                "median": statistics.median(numbers),
                "mode": statistics.mode(numbers) if len(set(numbers)) < len(numbers) else "No mode",
                "std_dev": statistics.stdev(numbers) if len(numbers) > 1 else 0,
                "variance": statistics.variance(numbers) if len(numbers) > 1 else 0,
                "min": min(numbers),
                "max": max(numbers),
                "range": max(numbers) - min(numbers)
            }
            
        elif operation == "trigonometry":
            # Apply trigonometric functions (assuming numbers are angles in degrees)
            angles_rad = [math.radians(x) for x in numbers]
            result = {
                "operation": "trigonometry",
                "input_angles_degrees": numbers,
                "sine_values": [round(math.sin(x), 4) for x in angles_rad],
                "cosine_values": [round(math.cos(x), 4) for x in angles_rad],
                "tangent_values": [round(math.tan(x), 4) for x in angles_rad],
                "sum_of_sines": round(sum(math.sin(x) for x in angles_rad), 4),
                "sum_of_cosines": round(sum(math.cos(x) for x in angles_rad), 4)
            }
            
        elif operation == "advanced":
            # Advanced mathematical operations
            result = {
                "operation": "advanced",
                "input_numbers": numbers,
                "square_roots": [round(math.sqrt(abs(x)), 4) for x in numbers],
                "logarithms": [round(math.log(x), 4) if x > 0 else "undefined" for x in numbers],
                "factorials": [math.factorial(int(x)) if x >= 0 and x == int(x) and x <= 20 else "too_large_or_invalid" for x in numbers],
                "powers_of_2": [round(2**x, 4) if x <= 20 else "too_large" for x in numbers],
                "fibonacci_at_positions": [fib(int(x)) if x >= 0 and x == int(x) and x <= 30 else "invalid" for x in numbers]
            }
        
        else:
            result = {"error": f"Unknown operation: {operation}"}
        
        state.action_result = result
        print(f"✅ Mathematical computation '{operation}' completed")
        
    except Exception as e:
        state.action_result = {"error": str(e), "operation": operation}
        print(f"❌ Mathematical computation failed: {e}")

def fib(n):
    """Calculate Fibonacci number efficiently."""
    if n <= 1:
        return n
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

# Test statistical analysis
print("\n📊 Testing statistical analysis:")
sample_data = [12, 15, 18, 22, 25, 28, 30, 15, 20, 25, 18, 22, 26, 19, 23]

with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> MATH_COMP
    MATH_COMP --> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "numbers": {"type": "getter", "dependency": "get_numbers"},
                    "math_operation": {"type": "getter", "dependency": "get_operation"},
                }
            },
            "MATH_COMP": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "MATH_COMP",
                "action": "math_computation_action",
                "output_data": ["action_result"],
            },
            "FINAL": {
                "output_data": ["action_result"],
            },
        },
        get_numbers=lambda state: sample_data,
        get_operation=lambda state: "statistics",
        math_computation_action=math_computation_action,
    )

await fsm.run_async()  # INIT -> MATH_COMP
await fsm.run_async()  # MATH_COMP -> FINAL

result = fsm.state_bag.get("action_result")
print("📋 Statistical Analysis Results:")
for key, value in result.items():
    if isinstance(value, float):
        print(f"   {key}: {value:.2f}")
    else:
        print(f"   {key}: {value}")

# Test trigonometric calculations
print("\n📐 Testing trigonometric calculations:")
angles = [0, 30, 45, 60, 90, 120, 180, 270, 360]

with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> MATH_COMP
    MATH_COMP --> FINAL
    """
) as builder:
    fsm2 = builder.build(
        {
            "INIT": {
                "input_data": {
                    "numbers": {"type": "getter", "dependency": "get_numbers"},
                    "math_operation": {"type": "getter", "dependency": "get_operation"},
                }
            },
            "MATH_COMP": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "MATH_COMP",
                "action": "math_computation_action",
                "output_data": ["action_result"],
            },
            "FINAL": {
                "output_data": ["action_result"],
            },
        },
        get_numbers=lambda state: angles,
        get_operation=lambda state: "trigonometry",
        math_computation_action=math_computation_action,
    )

await fsm2.run_async()  # INIT -> MATH_COMP
await fsm2.run_async()  # MATH_COMP -> FINAL

result2 = fsm2.state_bag.get("action_result")
print("📋 Trigonometric Results:")
print(f"   Angles: {result2.get('input_angles_degrees')}")
print(f"   Sine values: {result2.get('sine_values')}")
print(f"   Cosine values: {result2.get('cosine_values')}")
print(f"   Sum of sines: {result2.get('sum_of_sines')}")
print(f"   Sum of cosines: {result2.get('sum_of_cosines')}")

---

## Demo 5: Content Generation and Streaming

Use `PureActionState` to generate content and set up streaming for consumption by other states.


In [ ]:
print("=== Demo 5: Content Generation and Streaming ===")

# Define content generation action with streaming
async def content_generation_action(state):
    """Generate content and set up streaming."""
    
    # Get configuration from state bag
    content_type = state.machine.state_bag.get("content_type", "story")
    topic = state.machine.state_bag.get("topic", "adventure")
    length = state.machine.state_bag.get("length", "short")
    
    print(f"📝 Generating {length} {content_type} about {topic}")
    
    # Pre-generated content for different combinations
    content_templates = {
        ("story", "adventure", "short"): [
            "Once upon a time, in a land far away, ",
            "there lived a brave explorer named Alex. ",
            "One day, Alex discovered a mysterious map ",
            "that led to a hidden treasure. ",
            "With courage and determination, ",
            "Alex embarked on an incredible journey ",
            "through dangerous forests and treacherous mountains. ",
            "After many challenges, the treasure was found, ",
            "and Alex returned home as a hero."
        ],
        ("story", "mystery", "short"): [
            "The old mansion stood silent in the moonlight. ",
            "Detective Sarah approached the front door, ",
            "knowing that somewhere inside lay the answer ",
            "to the case that had puzzled her for weeks. ",
            "The door creaked open, revealing shadows ",
            "that seemed to dance in the darkness. ",
            "Step by step, clue by clue, ",
            "the truth slowly began to emerge, ",
            "until finally, the mystery was solved."
        ],
        ("poem", "nature", "short"): [
            "In forests deep where shadows play,\n",
            "And sunlight filters through the leaves,\n", 
            "The gentle breeze whispers today\n",
            "Of secrets that the earth achieves.\n\n",
            "Birds sing songs of ancient lore,\n",
            "While streams dance over polished stones,\n",
            "Nature's beauty, ever more,\n",
            "Touches hearts and calms our bones."
        ]
    }
    
    # Get content or generate default
    key = (content_type, topic, length)
    content_chunks = content_templates.get(key, [
        f"This is a {length} {content_type} about {topic}. ",
        "It demonstrates how PureActionState can generate content ",
        "and provide it as a stream for other states to consume. ",
        "The content is chunked to simulate real-time generation."
    ])
    
    # Create streaming function
    async def content_streamer():
        """Stream the generated content chunk by chunk."""
        for i, chunk in enumerate(content_chunks):
            # Simulate processing time
            await asyncio.sleep(0.1)
            yield f"chunk_{i+1}: {chunk}"
        
        # Add final metadata
        yield f"metadata: Generated {len(content_chunks)} chunks for {content_type}"
    
    # Set up results
    state.action_result = {
        "content_type": content_type,
        "topic": topic,
        "length": length,
        "total_chunks": len(content_chunks),
        "generation_timestamp": datetime.now().isoformat(),
        "status": "ready_for_streaming"
    }
    
    # Store streamer in state bag for consumption
    state.machine.state_bag["content_streamer"] = content_streamer()
    
    print(f"✅ Content generation completed, {len(content_chunks)} chunks ready")

# Test content generation with different types
test_scenarios = [
    {"content_type": "story", "topic": "adventure", "length": "short"},
    {"content_type": "story", "topic": "mystery", "length": "short"},
    {"content_type": "poem", "topic": "nature", "length": "short"}
]

for i, scenario in enumerate(test_scenarios, 1):
    print(f"\n📖 Test Scenario {i}: {scenario['content_type']} about {scenario['topic']}")
    
    with AgenticStateMachine.StateMachineBuilder(
        """
        INIT --> GENERATE
        GENERATE --> FINAL
        """
    ) as builder:
        fsm = builder.build(
            {
                "INIT": {
                    "input_data": {
                        "content_type": {"type": "getter", "dependency": "get_content_type"},
                        "topic": {"type": "getter", "dependency": "get_topic"},
                        "length": {"type": "getter", "dependency": "get_length"},
                    }
                },
                "GENERATE": {
                    "module_path": "gai.asm.states",
                    "class_name": "PureActionState",
                    "title": "GENERATE",
                    "action": "content_generation_action",
                    "output_data": ["action_result", "content_streamer"],
                },
                "FINAL": {
                    "output_data": ["action_result", "content_streamer"],
                },
            },
            get_content_type=lambda state: scenario["content_type"],
            get_topic=lambda state: scenario["topic"],
            get_length=lambda state: scenario["length"],
            content_generation_action=content_generation_action,
        )
    
    await fsm.run_async()  # INIT -> GENERATE
    await fsm.run_async()  # GENERATE -> FINAL
    
    # Display generation results
    result = fsm.state_bag.get("action_result")
    print(f"   ✅ Generated: {result.get('total_chunks')} chunks")
    print(f"   📅 Timestamp: {result.get('generation_timestamp')}")
    
    # Consume the stream
    streamer = fsm.state_bag.get("content_streamer")
    if streamer:
        print("   📺 Streaming content:")
        async for chunk in streamer:
            if chunk.startswith("metadata:"):
                print(f"   ℹ️  {chunk}")
            else:
                print(f"   📄 {chunk.strip()}")

---

## Demo 6: Chain Multiple PureActionStates

Demonstrate chaining multiple `PureActionState` instances to create a processing pipeline.


In [ ]:
print("=== Demo 6: Chain Multiple PureActionStates ===")

# Define multiple action functions for chaining
async def data_input_action(state):
    """First action: Generate input data."""
    raw_data = [
        {"id": 1, "value": 10, "category": "A"},
        {"id": 2, "value": 25, "category": "B"}, 
        {"id": 3, "value": 15, "category": "A"},
        {"id": 4, "value": 30, "category": "C"},
        {"id": 5, "value": 20, "category": "B"}
    ]
    
    state.action_result = {
        "stage": "input",
        "data": raw_data,
        "count": len(raw_data),
        "timestamp": datetime.now().isoformat()
    }
    
    # Store data for next stage
    state.machine.state_bag["pipeline_data"] = raw_data
    print(f"✅ Stage 1: Generated {len(raw_data)} input records")

async def data_transform_action(state):
    """Second action: Transform the data."""
    input_data = state.machine.state_bag.get("pipeline_data", [])
    
    # Transform: add computed fields and filter
    transformed_data = []
    for item in input_data:
        transformed_item = {
            **item,
            "value_doubled": item["value"] * 2,
            "value_squared": item["value"] ** 2,
            "is_high_value": item["value"] > 20
        }
        transformed_data.append(transformed_item)
    
    state.action_result = {
        "stage": "transform", 
        "data": transformed_data,
        "transformations_applied": ["doubled", "squared", "high_value_flag"],
        "count": len(transformed_data),
        "timestamp": datetime.now().isoformat()
    }
    
    # Store transformed data for next stage
    state.machine.state_bag["pipeline_data"] = transformed_data
    print(f"✅ Stage 2: Transformed {len(transformed_data)} records")

async def data_analyze_action(state):
    """Third action: Analyze the transformed data."""
    input_data = state.machine.state_bag.get("pipeline_data", [])
    
    # Perform analysis
    categories = {}
    total_value = 0
    high_value_count = 0
    
    for item in input_data:
        category = item["category"]
        value = item["value"]
        
        if category not in categories:
            categories[category] = {"count": 0, "total_value": 0, "avg_value": 0}
        
        categories[category]["count"] += 1
        categories[category]["total_value"] += value
        total_value += value
        
        if item["is_high_value"]:
            high_value_count += 1
    
    # Calculate averages
    for cat_data in categories.values():
        cat_data["avg_value"] = round(cat_data["total_value"] / cat_data["count"], 2)
    
    state.action_result = {
        "stage": "analyze",
        "total_records": len(input_data),
        "total_value": total_value,
        "average_value": round(total_value / len(input_data), 2),
        "high_value_count": high_value_count,
        "categories": categories,
        "timestamp": datetime.now().isoformat()
    }
    
    print(f"✅ Stage 3: Analyzed {len(input_data)} records across {len(categories)} categories")

async def data_output_action(state):
    """Fourth action: Generate final output report."""
    
    # Collect results from all previous stages
    input_result = state.machine.state_history[1]["output"].get("action_result", {})
    transform_result = state.machine.state_history[2]["output"].get("action_result", {})
    analyze_result = state.machine.state_history[3]["output"].get("action_result", {})
    
    # Generate comprehensive report
    report = {
        "pipeline_summary": {
            "stages_completed": 4,
            "total_processing_time": "simulated",
            "status": "success"
        },
        "input_stage": {
            "records_generated": input_result.get("count", 0),
            "timestamp": input_result.get("timestamp")
        },
        "transform_stage": {
            "records_processed": transform_result.get("count", 0),
            "transformations": transform_result.get("transformations_applied", []),
            "timestamp": transform_result.get("timestamp")
        },
        "analysis_stage": {
            "total_value": analyze_result.get("total_value", 0),
            "average_value": analyze_result.get("average_value", 0),
            "high_value_records": analyze_result.get("high_value_count", 0),
            "categories_found": len(analyze_result.get("categories", {})),
            "timestamp": analyze_result.get("timestamp")
        },
        "final_report_generated": datetime.now().isoformat()
    }
    
    state.action_result = report
    print(f"✅ Stage 4: Generated final pipeline report")

# Build and run the chained pipeline
print("\n🔗 Building data processing pipeline with 4 chained PureActionStates:")

with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> INPUT
    INPUT --> TRANSFORM
    TRANSFORM --> ANALYZE
    ANALYZE --> OUTPUT
    OUTPUT --> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {}
            },
            "INPUT": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "INPUT",
                "action": "data_input_action",
                "output_data": ["action_result"],
            },
            "TRANSFORM": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "TRANSFORM", 
                "action": "data_transform_action",
                "output_data": ["action_result"],
            },
            "ANALYZE": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "ANALYZE",
                "action": "data_analyze_action", 
                "output_data": ["action_result"],
            },
            "OUTPUT": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "OUTPUT",
                "action": "data_output_action",
                "output_data": ["action_result"],
            },
            "FINAL": {
                "output_data": ["action_result"],
            },
        },
        data_input_action=data_input_action,
        data_transform_action=data_transform_action,
        data_analyze_action=data_analyze_action,
        data_output_action=data_output_action,
    )

# Execute the pipeline step by step
print("\n🚀 Executing pipeline:")
await fsm.run_async()  # INIT -> INPUT
await fsm.run_async()  # INPUT -> TRANSFORM  
await fsm.run_async()  # TRANSFORM -> ANALYZE
await fsm.run_async()  # ANALYZE -> OUTPUT
await fsm.run_async()  # OUTPUT -> FINAL

# Display final results
final_result = fsm.state_bag.get("action_result")
print("\n📋 Final Pipeline Report:")
print(json.dumps(final_result, indent=2))

print("\n📊 Pipeline Execution Summary:")
for i, state_info in enumerate(fsm.state_history):
    state_name = state_info.get("state", "Unknown")
    print(f"   {i+1}. {state_name}")

print("\n🎉 Chained PureActionState demo completed successfully!")

---

## Summary

This notebook demonstrated various powerful uses of `PureActionState`:

### **Key Features Demonstrated:**

1. **Data Processing**: Statistical analysis and data transformation
2. **File Operations**: File creation, reading, and analysis  
3. **API Integration**: Simulated API calls with retry logic
4. **Mathematical Computations**: Statistics, trigonometry, and advanced math
5. **Content Generation**: Creating and streaming content
6. **Pipeline Processing**: Chaining multiple actions in sequence

### **Best Practices Shown:**

- ✅ **Error Handling**: Proper exception handling in action functions
- ✅ **State Communication**: Using state_bag to pass data between states
- ✅ **Result Storage**: Storing results in `action_result` for state output
- ✅ **Async Operations**: Proper async/await usage for non-blocking operations
- ✅ **Resource Cleanup**: Cleaning up temporary files and resources
- ✅ **Progress Reporting**: Providing feedback during action execution

### **Action Function Pattern:**
```python
async def my_action(state):
    # 1. Get input from state.machine.state_bag
    input_data = state.machine.state_bag.get("input_key")
    
    # 2. Perform the action/computation
    result = process_data(input_data)
    
    # 3. Store result for state output
    state.action_result = result
    
    # 4. Optionally store data for next states
    state.machine.state_bag["output_key"] = processed_data
```

`PureActionState` provides a flexible foundation for implementing custom business logic, data processing, and integration tasks within the Agentic State Machine framework.
